## Exercícios - WithColumn 1/2

### 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [2]:
!hdfs dfs -cat /user/manoel/data/exercises-data/juros_selic/juros_selic

"data";"valor"
"01/06/1986";"1,27"
"01/07/1986";"1,95"
"01/08/1986";"2,57"
"01/09/1986";"2,94"
"01/10/1986";"1,96"
"01/11/1986";"2,37"
"01/12/1986";"5,47"
"01/01/1987";"11,00"
"01/02/1987";"19,61"
"01/03/1987";"11,95"
"01/04/1987";"15,30"
"01/05/1987";"24,63"
"01/06/1987";"18,02"
"01/07/1987";"8,91"
"01/08/1987";"8,09"
"01/09/1987";"7,99"
"01/10/1987";"9,45"
"01/11/1987";"12,92"
"01/12/1987";"14,38"
"01/01/1988";"16,78"
"01/02/1988";"18,35"
"01/03/1988";"16,59"
"01/04/1988";"20,25"
"01/05/1988";"18,65"
"01/06/1988";"20,17"
"01/07/1988";"24,69"
"01/08/1988";"22,63"
"01/09/1988";"26,25"
"01/10/1988";"29,79"
"01/11/1988";"28,41"
"01/12/1988";"30,24"
"01/01/1989";"22,97"
"01/02/1989";"18,95"
"01/03/1989";"20,41"
"01/04/1989";"11,52"
"01/05/1989";"11,43"
"01/06/1989";"27,29"
"01/07/1989";"33,15"
"01/08/1989";"35,49"
"01/09/1989";"38,58"
"01/10/1989";"47,70"
"01/11/1989";"48,41"
"01/12/1989";"64,21"
"01/01/1990";"67,60"
"01/02/1990";"82,04"
"01/03/1990";"36,76"
"01/04/1990";"4,23"
"01/05/199

In [5]:
from pyspark.sql.functions import *

In [3]:
juros = spark.read.csv("/user/manoel/data/exercises-data/juros_selic/juros_selic", sep=";", header="true")

### 2. Alterar o formato do campo data para “MM/dd/yyyy”

In [4]:
juros.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [10]:
#juros_americano_unix = juros.withColumn("data", unix_timestamp(col("data"), "dd/MM/yyyy"))
#juros_americano = juros_americano_unix.withColumn("data", from_unixtime("data", "MM/dd/yyyy")).show()
# outra forma de fazer em uma unica linha.
juros_americano = juros.withColumn("data", from_unixtime(unix_timestamp(col("data"), "dd/MM/yyyy"), "MM/dd/yyyy"))
juros_americano.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|06/01/1986| 1,27|
|07/01/1986| 1,95|
|08/01/1986| 2,57|
+----------+-----+
only showing top 3 rows



### 3. Com uso da função from_unixtime crie o campo “ano_unix”, com a informação do ano do campo data

In [11]:
# usa-se este quando tiver um formato de data que precisa converter ele.
juros_unixtime = juros_americano.withColumn("ano_unix", from_unixtime(unix_timestamp(col("data"), "dd/MM/yyyy"), "yyyy"))
juros_unixtime.show(3)

+----------+-----+--------+
|      data|valor|ano_unix|
+----------+-----+--------+
|06/01/1986| 1,27|    1986|
|07/01/1986| 1,95|    1986|
|08/01/1986| 2,57|    1986|
+----------+-----+--------+
only showing top 3 rows



### 4. Com uso de substring crie o campo “ano_str”, com a informação do ano do campo data

In [12]:
# usa-se este quando tiver algo mais congelado.
juros_substring = juros_unixtime.withColumn("ano_str", substring(col("data"),7,4))
juros_substring.show(3)

+----------+-----+--------+-------+
|      data|valor|ano_unix|ano_str|
+----------+-----+--------+-------+
|06/01/1986| 1,27|    1986|   1986|
|07/01/1986| 1,95|    1986|   1986|
|08/01/1986| 2,57|    1986|   1986|
+----------+-----+--------+-------+
only showing top 3 rows



### 5. Com uso da função split crie o campo “ano_str”, com a informação do ano do campo data

In [13]:
# usa-se este quando tem algo preso previo nos dados.
juros_split = juros_substring.withColumn("ano_split", split(col("data"), "/").getItem(2))
juros_split.show(3)

+----------+-----+--------+-------+---------+
|      data|valor|ano_unix|ano_str|ano_split|
+----------+-----+--------+-------+---------+
|06/01/1986| 1,27|    1986|   1986|     1986|
|07/01/1986| 1,95|    1986|   1986|     1986|
|08/01/1986| 2,57|    1986|   1986|     1986|
+----------+-----+--------+-------+---------+
only showing top 3 rows



### 6. Salvar no hdfs /user/rodrigo/juros_selic_americano no formato CSV, incluindo o cabeçalho

In [14]:
juros_split.write.csv("/user/manoel/juros_selic", header="true")

In [16]:
!hdfs dfs -ls /user/manoel/juros_selic

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-12 23:19 /user/manoel/juros_selic/_SUCCESS
-rw-r--r--   2 root supergroup      13086 2022-04-12 23:19 /user/manoel/juros_selic/part-00000-03b1ad63-d4ac-4c68-b11f-35be10099942-c000.csv


In [17]:
!hdfs dfs -cat /user/manoel/juros_selic/part-00000-03b1ad63-d4ac-4c68-b11f-35be10099942-c000.csv

data,valor,ano_unix,ano_str,ano_split
06/01/1986,"1,27",1986,1986,1986
07/01/1986,"1,95",1986,1986,1986
08/01/1986,"2,57",1986,1986,1986
09/01/1986,"2,94",1986,1986,1986
10/01/1986,"1,96",1986,1986,1986
11/01/1986,"2,37",1986,1986,1986
12/01/1986,"5,47",1986,1986,1986
01/01/1987,"11,00",1987,1987,1987
02/01/1987,"19,61",1987,1987,1987
03/01/1987,"11,95",1987,1987,1987
04/01/1987,"15,30",1987,1987,1987
05/01/1987,"24,63",1987,1987,1987
06/01/1987,"18,02",1987,1987,1987
07/01/1987,"8,91",1987,1987,1987
08/01/1987,"8,09",1987,1987,1987
09/01/1987,"7,99",1987,1987,1987
10/01/1987,"9,45",1987,1987,1987
11/01/1987,"12,92",1987,1987,1987
12/01/1987,"14,38",1987,1987,1987
01/01/1988,"16,78",1988,1988,1988
02/01/1988,"18,35",1988,1988,1988
03/01/1988,"16,59",1988,1988,1988
04/01/1988,"20,25",1988,1988,1988
05/01/1988,"18,65",1988,1988,1988
06/01/1988,"20,17",1988,1988,1988
07/01/1988,"24,69",1988,1988,1988
08/01/1988,"22,63",1988,1988,1988
09/01/1988,"26,25",1988,1988,1988
10/01/1988,"29,79",19

## Exercícios - WithColumn 2/2

### 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
juros_selic = spark.read.csv("/user/manoel/data/exercises-data/juros_selic/juros_selic", header="true", sep=";")

### 2. Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor com a seguinte estrutura:

Anual 		Meses 		Valor médio 		Valor Mínimo 		Valor Máximo 

2019 		2 			00.00 				00.00 				00.00

2018 		12 			00.00 				00.00 				00.00

2017 		12 			00.00 				00.00 				00.00

... 		... 		00.00 				00.00 				00.00

1986 		2  			00.00 				00.00 				00.00

In [10]:
juros_ano = juros_selic.withColumn("ano", split(col("data"), "/").getItem(2))
juros_valor = juros_ano.withColumn("valor", regexp_replace(col("valor"),"\,",".").cast(FloatType()))
juros_valor.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)



In [20]:
juros_relatorio = juros_valor.groupBy("ano").agg(count("ano").alias("Meses"), format_number(avg("valor"), 2).alias("Valor médio"), min("valor").alias("Valor mínimo"), max("valor").alias("Valor máximo")).sort(desc("ano"))

### 3. Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc

In [21]:
juros_relatorio.write.orc("/user/manoel/relatorio_anual", compression="zlib")

In [22]:
!hdfs dfs -ls /user/manoel/relatorio_anual

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-13 00:18 /user/manoel/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        582 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00000-beb776ef-ef09-4745-a83b-0f60151a3b01-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00001-beb776ef-ef09-4745-a83b-0f60151a3b01-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00002-beb776ef-ef09-4745-a83b-0f60151a3b01-c000.zlib.orc
-rw-r--r--   2 root supergroup        572 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00003-beb776ef-ef09-4745-a83b-0f60151a3b01-c000.zlib.orc
-rw-r--r--   2 root supergroup        583 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00004-beb776ef-ef09-4745-a83b-0f60151a3b01-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2022-04-13 00:18 /user/manoel/relatorio_anual/part-00005-beb776ef-ef09-4745-a83b-0f60151a3b01-c0